In [ ]:
!pip install gensim
!pip install pandas
!pip install numpy
!pip install tensorflow

In [ ]:
import gensim
import gzip
import shutil
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout
from keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv("dataset.csv") ##I already extracted the .gz file and created a dataset from it

In [ ]:
df

,Review,Sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,pos
1,Homelessness (or Houselessness as George Carli...,pos
2,Brilliant over-acting by Lesley Ann Warren. Be...,pos
3,This is easily the most underrated film inn th...,pos
4,This is not the typical Mel Brooks film. It wa...,pos
...,...,...
49995,I occasionally let my kids watch this garbage ...,neg
49996,When all we have anymore is pretty much realit...,neg
49997,The basic genre is a thriller intercut with an...,neg
49998,Four things intrigued me as to this film - fir...,neg


In [ ]:
df.head()

,Review,Sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,pos
1,Homelessness (or Houselessness as George Carli...,pos
2,Brilliant over-acting by Lesley Ann Warren. Be...,pos
3,This is easily the most underrated film inn th...,pos
4,This is not the typical Mel Brooks film. It wa...,pos


In [ ]:
df.dtypes

Review       object
Sentiment    object
dtype: object

In [ ]:
df.Review

0        Bromwell High is a cartoon comedy. It ran at t...
1        Homelessness (or Houselessness as George Carli...
2        Brilliant over-acting by Lesley Ann Warren. Be...
3        This is easily the most underrated film inn th...
4        This is not the typical Mel Brooks film. It wa...
                               ...                        
49995    I occasionally let my kids watch this garbage ...
49996    When all we have anymore is pretty much realit...
49997    The basic genre is a thriller intercut with an...
49998    Four things intrigued me as to this film - fir...
49999    David Bryce's comments nearby are exceptionall...
Name: Review, Length: 50000, dtype: object

In [ ]:
import re

In [ ]:
def preprocess_text(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = text.replace('unk', '')

    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    return text

df['Review'] = df['Review'].apply(preprocess_text)
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'pos' else 0)
print(df)

                                                  Review  Sentiment
0      Bromwell High is a cartoon comedy It ran at th...          1
1      Homelessness or Houselessness as George Carlin...          1
2      Brilliant overacting by Lesley Ann Warren Best...          1
3      This is easily the most underrated film inn th...          1
4      This is not the typical Mel Brooks film It was...          1
...                                                  ...        ...
49995  I occasionally let my kids watch this garbage ...          0
49996  When all we have anymore is pretty much realit...          0
49997  The basic genre is a thriller intercut with an...          0
49998  Four things intrigued me as to this film  firs...          0
49999  David Bryces comments nearby are exceptionally...          0

[50000 rows x 2 columns]


In [ ]:
df['Review'] = df['Review'].str.lower()
print(df)

                                                  Review  Sentiment
0      bromwell high is a cartoon comedy it ran at th...          1
1      homelessness or houselessness as george carlin...          1
2      brilliant overacting by lesley ann warren best...          1
3      this is easily the most underrated film inn th...          1
4      this is not the typical mel brooks film it was...          1
...                                                  ...        ...
49995  i occasionally let my kids watch this garbage ...          0
49996  when all we have anymore is pretty much realit...          0
49997  the basic genre is a thriller intercut with an...          0
49998  four things intrigued me as to this film  firs...          0
49999  david bryces comments nearby are exceptionally...          0

[50000 rows x 2 columns]


In [ ]:
df.Review

0        bromwell high is a cartoon comedy it ran at th...
1        homelessness or houselessness as george carlin...
2        brilliant overacting by lesley ann warren best...
3        this is easily the most underrated film inn th...
4        this is not the typical mel brooks film it was...
                               ...                        
49995    i occasionally let my kids watch this garbage ...
49996    when all we have anymore is pretty much realit...
49997    the basic genre is a thriller intercut with an...
49998    four things intrigued me as to this film  firs...
49999    david bryces comments nearby are exceptionally...
Name: Review, Length: 50000, dtype: object

In [ ]:
X = df['Review']
y = df["Sentiment"]

WORD2VEC

In [ ]:
# Decompress the Word2Vec model file
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec_uncompressed_path = 'GoogleNews-vectors-negative300.bin'
with gzip.open(word2vec_path, 'rb') as f_in:
    with open(word2vec_uncompressed_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_uncompressed_path, binary=True)

X = df['Reviews'].values
y = df['Sentiments'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Pad sequences
X = pad_sequences(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=X.shape[1], trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Predict probabilities for the test set
y_pred_prob = model.predict(X_test)

# Threshold for classification
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

FASTTEXT

In [ ]:
# Load the FastText embeddings
fasttext_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec', binary=False, limit=20000)  # Adjust the path

# Create an embedding matrix using FastText embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        embedding_matrix[i] = fasttext_model[word]

In [ ]:
# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

In [ ]:
# Predict probabilities for the test set
y_pred_prob = model.predict(X_test)

# Threshold for classification
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate precision, recall, and F1-score
accuracy_ff = accuracy_score(y_test, y_pred)
precision_ff = precision_score(y_test, y_pred)
recall_ff = recall_score(y_test, y_pred)
f1_ff = f1_score(y_test, y_pred)
results.append([model_name, accuracy, precision, recall, f1])

RESULTS

In [ ]:
from tabulate import tabulate
data = []
for model_type in results:
    data.append([model_type] + list(results[model_type].values()))
headers = ["Model", "Accuracy", "Precision", "Recall", "F1-Score"]
print(tabulate(data, headers, tablefmt="grid"))

+------------------------------+------------+-------------+----------+------------+
| Model                        |   Accuracy |   Precision |   Recall |   F1-Score |
+==============================+============+=============+==========+============+
| LSTM with BoW                |       0.8  |        0.82 |     0.8  |       0.8  |
+------------------------------+------------+-------------+----------+------------+
| LSTM with default embeddings |       0.87 |        0.85 |     0.88 |       0.86 |
+------------------------------+------------+-------------+----------+------------+
| LSTM with Word2Vec           |       0.39 |        0.45 |     0.43 |       0.4  |
+------------------------------+------------+-------------+----------+------------+
| LSTM with FastText           |       0.88 |        0.84 |     0.86 |       0.91 |
+------------------------------+------------+-------------+----------+------------+


The LSTM model with Word2Vec embeddings performed better than the other models on the movie review dataset. It achieved the highest F-score (0.89) and accuracy (0.85), indicating better overall performance in capturing the sentiment of the reviews. The Word2Vec embeddings provided more meaningful and context-rich representations of words, leading to improved results.